# Loading Your API Keys

In [1]:
import json
with open('/Users/Rashad/.secret/yelp_api.json') as c: #change the path to match YOUR path!!
    login = json.load(c)
login.keys()

dict_keys(['client-id', 'api-key'])

# Use the correct key of your login dictionary for your task

In [2]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

# Test API key by performing a query.

In [3]:
# Quick Test Query
results = yelp_api.search_query(location='Baltimore, MD',
                                       term='food')
print(type(results))
results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

---

# Using YelpAPI to search for data using Yelp's [api search methods](https://github.com/lanl/yelpapi)

* Autocomplete API - autocomplete_query(...)
* Business API - business_query(...)
* Business Match API - business_match_query(...)
* Event Lookup API - event_lookup_query(...)
* Event Search API - event_search_query(...)
* Featured Event API - featured_event_query(...)
* Phone Search API - phone_search_query(...)
* Reviews API - reviews_query(...)
* Search API - search_query(...)
* Transaction Search API - transaction_search_query(...)

In [4]:
# HELP FUNCTION TO SEE WHAT'S REQUIRED OF THE `search_query()` function.
help(yelp_api.search_query)

Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



In [5]:
import pandas as pd

biz = pd.DataFrame(results['businesses'])
biz.head(2) 

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,tf6-VbiMWfnujYL3l5gAwQ,café-dear-leon-baltimore,Café Dear Leon,https://s3-media2.fl.yelpcdn.com/bphoto/e7Ke64...,False,https://www.yelp.com/biz/caf%C3%A9-dear-leon-b...,250,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",5.0,"{'latitude': 39.279914, 'longitude': -76.57432}",[delivery],$$,"{'address1': '2929 O'Donnell St', 'address2': ...",+16673122977,(667) 312-2977,4424.463723
1,g1ANu-RMrc_PsI_5ggn6ug,ovenbird-bakery-baltimore,Ovenbird Bakery,https://s3-media2.fl.yelpcdn.com/bphoto/0GmB8e...,False,https://www.yelp.com/biz/ovenbird-bakery-balti...,192,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",5.0,"{'latitude': 39.286596, 'longitude': -76.600793}",[delivery],$,"{'address1': '300 S Exeter St', 'address2': ''...",+14438859998,(443) 885-9998,2620.918430


# Pagination and "offset"

In [6]:
results['total']

5500

In [7]:
len(results['businesses'])

20

The Yelp API will only return a "page" of 20 results at a time.

* If we want to get the next page of results, we will perform another API call, but we will add an additional argument called "offset."
    * The offset is what # result to use as the FIRST result for the page.
    * If we had 20 businesses in our first result, we would want to add an offset of 20.

In [8]:
# add offset to our original api call
results = yelp_api.search_query(location='Baltimore, MD',
                                term='food',
                                offset = 20)

### Now check the results...

In [9]:
biz20 = pd.DataFrame(results['businesses'])
biz20.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8-cBsSXmLaGK6qCD0tmcZg,the-food-market-baltimore,The Food Market,https://s3-media4.fl.yelpcdn.com/bphoto/8vVCxz...,False,https://www.yelp.com/biz/the-food-market-balti...,1361,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 39.3310122069348, 'longitude': -7...",[delivery],$$,"{'address1': '1017 W 36th St', 'address2': '',...",+14103660606,(410) 366-0606,3186.763707
1,ieS_5zqxDHcWMCm8BKUYbg,thames-street-oyster-house-baltimore,Thames Street Oyster House,https://s3-media4.fl.yelpcdn.com/bphoto/SLti9z...,False,https://www.yelp.com/biz/thames-street-oyster-...,3072,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 39.28214, 'longitude': -76.59162}",[delivery],$$$,"{'address1': '1728 Thames St', 'address2': '',...",+14434497726,(443) 449-7726,3377.766396


# Now you can combine the results (so far) into one dataframe using pd.concat()

In [10]:
## concatenate the previous results and new results. 
businesses = pd.concat([biz, biz20], ignore_index=True)

display(businesses.head(3), businesses.tail(3))

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,tf6-VbiMWfnujYL3l5gAwQ,café-dear-leon-baltimore,Café Dear Leon,https://s3-media2.fl.yelpcdn.com/bphoto/e7Ke64...,False,https://www.yelp.com/biz/caf%C3%A9-dear-leon-b...,250,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",5.0,"{'latitude': 39.279914, 'longitude': -76.57432}",[delivery],$$,"{'address1': '2929 O'Donnell St', 'address2': ...",+16673122977,(667) 312-2977,4424.463723
1,g1ANu-RMrc_PsI_5ggn6ug,ovenbird-bakery-baltimore,Ovenbird Bakery,https://s3-media2.fl.yelpcdn.com/bphoto/0GmB8e...,False,https://www.yelp.com/biz/ovenbird-bakery-balti...,192,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",5.0,"{'latitude': 39.286596, 'longitude': -76.600793}",[delivery],$,"{'address1': '300 S Exeter St', 'address2': ''...",+14438859998,(443) 885-9998,2620.918430
2,wVrYzb_zfcQGaeZQ63Cs-g,good-neighbor-baltimore,good neighbor,https://s3-media3.fl.yelpcdn.com/bphoto/c52mvc...,False,https://www.yelp.com/biz/good-neighbor-baltimo...,117,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 39.3347706280897, 'longitude': -7...",[delivery],$$,"{'address1': '3827 Falls Rd', 'address2': '', ...",+14436278919,(443) 627-8919,3670.986380


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
37,SvVrpe8VqglIRNhfOo1_1A,iron-rooster-baltimore,Iron Rooster,https://s3-media2.fl.yelpcdn.com/bphoto/ZV8WyY...,False,https://www.yelp.com/biz/iron-rooster-baltimor...,1033,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 39.2764795638087, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '3721 Boston St', 'address2': '',...",+14107622100,(410) 762-2100,5239.549640
38,rtJyAgZTFkPigOSXdvD-HA,mama-rosa-grill-baltimore-baltimore,Mama Rosa Grill - Baltimore,https://s3-media1.fl.yelpcdn.com/bphoto/d3AhAe...,False,https://www.yelp.com/biz/mama-rosa-grill-balti...,45,"[{'alias': 'filipino', 'title': 'Filipino'}]",4.5,"{'latitude': 39.250009, 'longitude': -76.6392977}","[delivery, pickup]",$$,"{'address1': '3321 Annapolis Rd', 'address2': ...",+14104699600,(410) 469-9600,7144.171066
39,BctKgpeSMajSnqAC4tJ6hA,yeeros-baltimore,Yeeros,https://s3-media4.fl.yelpcdn.com/bphoto/CuB6yy...,False,https://www.yelp.com/biz/yeeros-baltimore?adju...,62,"[{'alias': 'greek', 'title': 'Greek'}]",5.0,"{'latitude': 39.2765, 'longitude': -76.61351}","[delivery, pickup]",NaN,"{'address1': '17 E Cross St', 'address2': None...",+16672604998,(667) 260-4998,3668.718621


In [11]:
businesses.to_json(orient='records')

'[{"id":"tf6-VbiMWfnujYL3l5gAwQ","alias":"caf\\u00e9-dear-leon-baltimore","name":"Caf\\u00e9 Dear Leon","image_url":"https:\\/\\/s3-media2.fl.yelpcdn.com\\/bphoto\\/e7Ke64vUi9K6R3FQA9yczw\\/o.jpg","is_closed":false,"url":"https:\\/\\/www.yelp.com\\/biz\\/caf%C3%A9-dear-leon-baltimore?adjust_creative=eXgc5GOsWJUbUzVXXSFHSQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=eXgc5GOsWJUbUzVXXSFHSQ","review_count":250,"categories":[{"alias":"cafes","title":"Cafes"},{"alias":"bakeries","title":"Bakeries"},{"alias":"desserts","title":"Desserts"}],"rating":5.0,"coordinates":{"latitude":39.279914,"longitude":-76.57432},"transactions":["delivery"],"price":"$$","location":{"address1":"2929 O\'Donnell St","address2":"","address3":null,"city":"Baltimore","zip_code":"21224","country":"US","state":"MD","display_address":["2929 O\'Donnell St","Baltimore, MD 21224"]},"phone":"+16673122977","display_phone":"(667) 312-2977","distance":4424.4637232493},{"id":"g1ANu-RMrc_PsI_5ggn6ug","a